In [1]:
%pylab inline


Populating the interactive namespace from numpy and matplotlib


In [2]:
import pandas as pd
filepath = '/Users/seeni-2328/Documents/Seeni/_zoho_products/sdp/datasets/Incident_Category/sdp-requests/results/1_2014_04_10_To_2015_06_14_39849.csv'
p_df = pd.read_csv(filepath)

In [3]:
p_df.dtypes

CATEGORY          object
CREATED_TIME      object
DESCRIPTION       object
ID                 int64
KEYWORD           object
REQUESTER_NAME    object
SUBJECT           object
SUB_CATEGORY      object
TECHNICIAN        object
dtype: object

In [4]:
p_df['CONTENT'] = p_df['SUBJECT'].map(str) + ". " + p_df['DESCRIPTION']

In [5]:
p_df.head()

,CATEGORY,CREATED_TIME,DESCRIPTION,ID,KEYWORD,REQUESTER_NAME,SUBJECT,SUB_CATEGORY,TECHNICIAN,CONTENT
0,-,2015-05-08 19:10:57,Four candidates from Annna University are join...,1387292,"['need', 'estancia', 'developer machine', 'kin...",revathi.pk,Need 4 Machines @ Estancia,-,Deepak K,Need 4 Machines @ Estancia. Four candidates fr...
1,-,2015-01-22 11:28:26,"Chat and Group Email,Hi Team, While doing the ...",1328757,"['convention', 'respective email owner', 'emai...",Kamala Kannan Subramani,Re: Standard team naming conventions in Connec...,-,Parthasarathy K,Re: Standard team naming conventions in Connec...
2,-,2014-05-14 14:52:00,"As discussed earlier, we are facing the below ...",1186244,"['push failure', 'abort http error', 'bad requ...",Petchimuthu Nallakannu,Hg pull/push Failure in integ-toolsnew,-,Parthasarathy K,Hg pull/push Failure in integ-toolsnew. As dis...
3,-,2014-04-10 12:45:43,We are using cgi-bin functionality in our ME s...,1170307,"['site server', 'cgi script usage', 'late vers...",Kamala Kannan Subramani,Disable cgi-bin in ME site servers,-,Parthasarathy K,Disable cgi-bin in ME site servers. We are usi...
4,Alerts,2015-06-14 06:09:48,This is an auto generated e-mail triggered bec...,1403736,"['alert', 'application manager', 'zoho urls', ...",sv2-appmanager,Alert from the Applications Manager - [ Zoho U...,-,Dhayanathi Vijayakumar,Alert from the Applications Manager - [ Zoho U...


In [6]:
docs = array(p_df['CONTENT'])

In [7]:
len(docs)

29155

In [8]:
# raw tsne parameters 
p_df.count()

CATEGORY          29155
CREATED_TIME      29155
DESCRIPTION       29155
ID                29155
KEYWORD           29155
REQUESTER_NAME    29155
SUBJECT           29155
SUB_CATEGORY      29155
TECHNICIAN        29155
CONTENT           29155
dtype: int64

In [9]:
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer

def docs_preprocessor(docs):
    tokenizer = RegexpTokenizer(r'\w+')
    for idx in range(len(docs)):
        docs[idx] = docs[idx].lower()  # Convert to lowercase.
        docs[idx] = tokenizer.tokenize(docs[idx])  # Split into words.

    # Remove numbers, but not words that contain numbers.
    docs = [[token for token in doc if not token.isdigit()] for doc in docs]

    # Remove words that are only one character.
    docs = [[token for token in doc if len(token) > 3] for doc in docs]

    # Lemmatize all words in documents.
    lemmatizer = WordNetLemmatizer()
    docs = [[lemmatizer.lemmatize(token) for token in doc] for doc in docs]

    return docs


In [10]:
docs = docs_preprocessor(docs)

In [11]:
print(docs[0][0])

need


In [12]:
from gensim.models import Phrases
# Add bigrams and trigrams to docs (only ones that appear 10 times or more).
bigram = Phrases(docs, min_count=10)
trigram = Phrases(bigram[docs])

for idx in range(len(docs)):
    for token in bigram[docs[idx]]:
        if '_' in token:
            # Token is a bigram, add to document.
            docs[idx].append(token)
    for token in trigram[docs[idx]]:
        if '_' in token:
            # Token is a bigram, add to document.
            docs[idx].append(token)

/usr/local/lib/python3.6/site-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [13]:
from gensim.corpora import Dictionary

# Create a dictionary representation of the documents.
dictionary = Dictionary(docs)
print('Number of unique words in initital documents:', len(dictionary))

# Filter out words that occur less than 10 documents, or more than 20% of the documents.
dictionary.filter_extremes(no_below=10, no_above=0.2)
print('Number of unique words after removing rare and common words:', len(dictionary))

Number of unique words in initital documents: 15548
Number of unique words after removing rare and common words: 5515


In [14]:
corpus = [dictionary.doc2bow(doc) for doc in docs]
print('Number of unique tokens: %d' % len(dictionary))
print('Number of documents: %d' % len(corpus))


Number of unique tokens: 5515
Number of documents: 29155


In [15]:
from gensim.models import LdaModel
# Set training parameters.
num_topics = 10
chunksize = 500 # size of the doc looked at every pass
passes = 20 # number of passes through documents
iterations = 400
eval_every = 1  # Don't evaluate model perplexity, takes too much time.

# Make a index to word dictionary.
temp = dictionary[0]  # This is only to "load" the dictionary.
id2word = dictionary.id2token

%time model = LdaModel(corpus=corpus, id2word=id2word, chunksize=chunksize, \
                       alpha='auto', eta='auto', \
                       iterations=iterations, num_topics=num_topics, \
                       passes=passes, eval_every=eval_every)


/usr/local/lib/python3.6/site-packages/gensim/models/ldamodel.py:1023: RuntimeWarning: divide by zero encountered in log
  diff = np.log(self.expElogbeta)


CPU times: user 7min 21s, sys: 35 s, total: 7min 56s
Wall time: 5min 17s


In [16]:
import pyLDAvis.gensim
pyLDAvis.enable_notebook()


In [17]:
pyLDAvis.gensim.prepare(model, corpus, dictionary)

/usr/local/lib/python3.6/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3      0.323363 -0.041939       1        1  17.347071
4     -0.096128 -0.138754       2        1  13.327196
8      0.321546 -0.084526       3        1  12.282809
1     -0.040851 -0.048635       4        1  11.531379
9     -0.181351  0.004536       5        1  10.926612
0     -0.217275 -0.258309       6        1  10.247433
5     -0.103473  0.297386       7        1   7.775932
2     -0.052728  0.111391       8        1   5.610274
7      0.013559 -0.073991       9        1   5.561914
6      0.033337  0.232838      10        1   5.389380, topic_info=     Category          Freq                      Term         Total  loglift  \
term                                                                           
1785  Default  56177.000000         webmaster_server3  56177.000000  30.0000   
4907  Default  44735.000000        forwarded_sysadmin  44735.000000  29.0000   
113   Default  21773.000000                   manager  21773.000000  28.0000   
67    Default  31760.000000                   request  31760.000000  27.0000   
1739  Default  15299.000000                 forwarded  15299.000000  26.0000   
868   Default  12028.000000                     mysql  12028.000000  25.0000   
4908  Default  14915.000000                   granted  14915.000000  24.0000   
4909  Default  14915.000000           granted_manager  14915.000000  23.0000   
1694  Default  13071.000000         webmaster_server4  13071.000000  22.0000   
104   Default  12599.000000            auto_generated  12599.000000  21.0000   
737   Default  10328.000000                  sysadmin  10328.000000  20.0000   
1109  Default  10698.000000                 webmaster  10698.000000  19.0000   
1819  Default   7929.000000                  computer   7929.000000  18.0000   
425   Default  12815.000000                       min  12815.000000  17.0000   
1780  Default  10155.000000                  database  10155.000000  16.0000   
437   Default  12338.000000                       sec  12338.000000  15.0000   
12    Default   8699.000000                   machine   8699.000000  14.0000   
107   Default   9240.000000           being_monitored   9240.000000  13.0000   
3956  Default   8945.000000  connected_administration   8945.000000  12.0000   
50    Default  17372.000000                      with  17372.000000  11.0000   
5065  Default  10163.000000       machine_requirement  10163.000000  10.0000   
116   Default   8138.000000         triggered_because   8138.000000   9.0000   
1783  Default   7865.000000                   server3   7865.000000   8.0000   
960   Default   7750.000000                      file   7750.000000   7.0000   
893   Default   9753.000000                     match   9753.000000   6.0000   
885   Default   9674.000000                configured   9674.000000   5.0000   
561   Default  10654.000000                  resolved  10654.000000   4.0000   
111   Default   7701.000000          following_entity   7701.000000   3.0000   
651   Default   8891.000000                    joinee   8891.000000   2.0000   
304   Default  10299.000000                       doe  10299.000000   1.0000   
...       ...           ...                       ...           ...      ...   
727   Topic10    673.479858                   service    673.697571   2.9204   
289   Topic10    671.161865                     about    671.379517   2.9204   
5464  Topic10    629.984070             need_restrict    630.196289   2.9204   
300   Topic10    605.845642                      copy    606.062378   2.9204   
83    Topic10    587.349854                      form    587.564636   2.9204   
297   Topic10    567.718689                   control    567.935242   2.9204   
312   Topic10    563.532776             open_detailed    563.748840   2.9204   
1012  Topic10    558.871216                      send    559.087585   2.9204   
334   Topic10    554.588623  

In [18]:
from sklearn.metrics.pairwise import cosine_similarity

p_df['tokenz'] = docs

docs1 = p_df['tokenz'].apply(lambda l: l[:int0(len(l)/2)])
docs2 = p_df['tokenz'].apply(lambda l: l[int0(len(l)/2):])


In [19]:
corpus1 = [dictionary.doc2bow(doc) for doc in docs1]
corpus2 = [dictionary.doc2bow(doc) for doc in docs2]

# Using the corpus LDA model tranformation
lda_corpus1 = model[corpus1]
lda_corpus2 = model[corpus2]
for i in range(10) :
    print(lda_corpus2[i])

[(2, 0.15329018), (4, 0.013668834), (5, 0.7970847)]
[(0, 0.046187244), (1, 0.30878976), (2, 0.19794795), (6, 0.3755667), (9, 0.046810642)]
[(0, 0.7056925), (2, 0.11441387), (5, 0.043524113), (6, 0.078372784), (8, 0.04005619)]
[(6, 0.31719252), (8, 0.6009302), (9, 0.049474746)]
[(1, 0.7695381), (4, 0.012061659), (5, 0.010266001), (6, 0.18414973)]
[(1, 0.76953816), (4, 0.012061659), (5, 0.010266001), (6, 0.18414965)]
[(0, 0.89489037), (1, 0.011674195), (2, 0.0100216335), (4, 0.022771504), (5, 0.019381437), (6, 0.012967137)]
[(0, 0.5506706), (4, 0.018634906), (5, 0.3791743), (6, 0.010611569)]
[(4, 0.013668833), (5, 0.9443555)]
[(3, 0.46474403), (4, 0.015795954), (5, 0.47458574)]


In [20]:
from collections import OrderedDict
def get_doc_topic_dist(model, corpus, kwords=False):
    
    '''
    LDA transformation, for each doc only returns topics with non-zero weight
    This function makes a matrix transformation of docs in the topic space.
    '''
    top_dist =[]
    keys = []

    for d in corpus:
        tmp = {i:0 for i in range(num_topics)}
#         print(tmp)
        tmp.update(dict(model[d]))
#         print(dict(model[d]))
#         print(tmp)
        vals = list(OrderedDict(tmp).values())
        top_dist += [array(vals)]
        if kwords:
            keys += [array(vals).argmax()]
#         print("data : {}, top_dist : {}, keys : {}\n".format(d,vals, array(vals).argmax()))
    
#     print("top_dist : {}, keys : {}".format(top_dist,keys))

    return array(top_dist), keys

In [22]:
top_dist1, _ = get_doc_topic_dist(model, lda_corpus1)
top_dist2, _ = get_doc_topic_dist(model, lda_corpus2)

print("Intra similarity: cosine similarity for corresponding parts of a doc(higher is better):")
print(np.mean([cosine_similarity(c1.reshape(1, -1), c2.reshape(1, -1)) for c1,c2 in zip(top_dist1, top_dist2)]))

random_pairs = np.random.randint(0, len(p_df['CONTENT']), size=(400, 2))

print("Inter similarity: cosine similarity between random parts (lower is better):")
print(np.mean([cosine_similarity(top_dist1[i[0]].reshape(1, -1), top_dist2[i[1]].reshape(1, -1)) for i in random_pairs]))

Intra similarity: cosine similarity for corresponding parts of a doc(higher is better):
0.97196215
Inter similarity: cosine similarity between random parts (lower is better):
0.8958623


In [25]:
def explore_topic(lda_model, topic_number, topn, output=True):
    """
    accept a ldamodel, atopic number and topn vocabs of interest
    prints a formatted list of the topn terms
    """
    terms = []
    for term, frequency in lda_model.show_topic(topic_number, topn=topn):
        terms += [term]
        if output:
            print(u'{:20} {:.3f}'.format(term, round(frequency, 3)))
    
    return terms

In [26]:
topic_summaries = []
print(u'{:20} {}'.format(u'term', u'frequency') + u'\n')
for i in range(num_topics):
    print('Topic '+str(i)+' |---------------------\n')
    tmp = explore_topic(model,topic_number=i, topn=10, output=True )
#     print(tmp[:5])
    topic_summaries += [tmp[:5]]
    print


term                 frequency

Topic 0 |---------------------

min                  0.034
sec                  0.033
with                 0.032
resolved             0.030
match                0.027
configured           0.027
doe                  0.027
could_resolve        0.022
california           0.018
downtime_chart       0.012
Topic 1 |---------------------

webmaster_server3    0.166
webmaster_server4    0.039
webmaster            0.032
mysql                0.031
database             0.030
system               0.025
server3              0.023
file                 0.023
alias                0.021
joinee               0.020
Topic 2 |---------------------

connected_administration 0.032
machine_requirement  0.030
computer             0.026
tuesday_december     0.023
name                 0.021
server               0.020
time                 0.019
december             0.014
managed              0.013
zohocorp             0.013
Topic 3 |---------------------

forwarded_sysadmin   0.097

In [23]:
top_labels = {0: 'Statistics', 1:'Numerical Analysis', 2:'Online Learning', 3:'Deep Learning'}

In [24]:
import re
import nltk

from nltk.corpus import stopwords

stops = set(stopwords.words('english'))

def paper_to_wordlist( paper, remove_stopwords=True ):
    '''
        Function converts text to a sequence of words,
        Returns a list of words.
    '''
    lemmatizer = WordNetLemmatizer()
    # 1. Remove non-letters
    paper_text = re.sub("[^a-zA-Z]"," ", paper)
    # 2. Convert words to lower case and split them
    words = paper_text.lower().split()
    # 3. Remove stop words
    words = [w for w in words if not w in stops]
    # 4. Remove short words
    words = [t for t in words if len(t) > 2]
    # 5. lemmatizing
    words = [nltk.stem.WordNetLemmatizer().lemmatize(t) for t in words]

    return(words)

In [25]:
from sklearn.feature_extraction.text import TfidfVectorizer

tvectorizer = TfidfVectorizer(input='content', analyzer = 'word', lowercase=True, stop_words='english',\
                                  tokenizer=paper_to_wordlist, ngram_range=(1, 3), min_df=40, max_df=0.20,\
                                  norm='l2', use_idf=True, smooth_idf=True, sublinear_tf=True)

dtm = tvectorizer.fit_transform(p_df['PaperText']).toarray()

/usr/local/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:286: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['le'] not in stop_words.
  sorted(inconsistent))


In [26]:
top_dist =[]
for d in corpus:
    tmp = {i:0 for i in range(num_topics)}
    tmp.update(dict(model[d]))
    vals = list(OrderedDict(tmp).values())
    top_dist += [array(vals)]

In [27]:
top_dist, lda_keys= get_doc_topic_dist(model, corpus, True)
features = tvectorizer.get_feature_names()

In [28]:
top_ws = []
for n in range(len(dtm)):
    inds = int0(argsort(dtm[n])[::-1][:4])
    tmp = [features[i] for i in inds]
    
    top_ws += [' '.join(tmp)]
    
p_df['Text_Rep'] = pd.DataFrame(top_ws)
p_df['clusters'] = pd.DataFrame(lda_keys)
p_df['clusters'].fillna(10, inplace=True)

cluster_colors = {0: 'blue', 1: 'green', 2: 'yellow', 3: 'red', 4: 'skyblue', 5:'salmon', 6:'orange', 7:'maroon', 8:'crimson', 9:'black', 10:'gray'}

p_df['colors'] = p_df['clusters'].apply(lambda l: cluster_colors[l])

In [29]:
from sklearn.manifold import TSNE
tsne = TSNE(n_components=2)
X_tsne = tsne.fit_transform(top_dist)

In [30]:
p_df['X_tsne'] =X_tsne[:, 0]
p_df['Y_tsne'] =X_tsne[:, 1]

In [37]:
from bokeh.plotting import figure, show, output_notebook, save#, output_file
from bokeh.models import HoverTool, value, LabelSet, Legend, ColumnDataSource
output_notebook()

Loading BokehJS ...

In [38]:
source = ColumnDataSource(dict(
    x=p_df['X_tsne'],
    y=p_df['Y_tsne'],
    color=p_df['colors'],
    label=p_df['clusters'].apply(lambda l: top_labels[l]),
#     msize= p_df['marker_size'],
    topic_key= p_df['clusters'],
    title= p_df[u'Title'],
    content = p_df['Text_Rep']
))

In [39]:
title = 'T-SNE visualization of topics'

plot_lda = figure(plot_width=1000, plot_height=600,
                     title=title, tools="pan,wheel_zoom,box_zoom,reset,hover,previewsave",
                     x_axis_type=None, y_axis_type=None, min_border=1)

plot_lda.scatter(x='x', y='y', legend='label', source=source,
                 color='color', alpha=0.8, size=10)#'msize', )

# hover tools
hover = plot_lda.select(dict(type=HoverTool))
hover.tooltips = {"content": "Title: @title, KeyWords: @content - Topic: @topic_key "}
plot_lda.legend.location = "top_left"

show(plot_lda)

#save the plot
# save(plot_lda, '{}.html'.format(title))